In [1]:
import pandas as pd
import csv
import time
from sklearn.model_selection import train_test_split


# check version number
import imblearn
print(imblearn.__version__)

from collections import Counter
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder


#warnings.filterwarnings('ignore')

0.8.0



Bad key "text.kerning_factor" on line 4 in
/Users/apple/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
# load data
train = pd.read_csv("train.csv", header = None)

test = pd.read_csv("test.csv", header = None)

#print(test.info())
#train.info()


In [3]:
# minor_size is number of classes less than 500 instances
minor_size = 0
grouped = train.groupby(train.columns[-1])
for k,group in grouped:
        if len(group.index) < 500:
            minor_size += 1
        #print ("Class label: ",k,end='')
        #print("  Number:",len(group.index), end='')
        #print(" (%.3f%%)" %  float(len(group.index)/len(train)*100))
minor_size

39

## Split Data

In [4]:
# exclude sample ID (1st column)
x_train_valid = train.iloc[:,1:-1].to_numpy()
y_train_valid = train.iloc[:,-1].to_numpy()

#split a validation set for hyper-parameter tuning
split_ratio = 0.2
x_train, x_val, y_train, y_val = train_test_split(x_train_valid, y_train_valid, test_size=split_ratio, random_state=1,stratify = y_train_valid) 

x_test = test.iloc[:,1:].to_numpy()
#y_test = test.iloc[:,-1].to_numpy()

### Apply SMOTE

In [6]:
from imblearn.over_sampling import ADASYN, BorderlineSMOTE, SMOTE

# we only oversample during training
ada = ADASYN(random_state = 42, sampling_strategy = 'minority', n_neighbors=5)
bdsmt = BorderlineSMOTE(random_state = 42)
smt = SMOTE(random_state = 42)

x_ada_train = x_train.copy()
y_ada_train = y_train.copy()

x_bdsmt_train = x_train.copy()
y_bdsmt_train = y_train.copy()

x_smt_train = x_train.copy()
y_smt_train = y_train.copy()

# ada failed to resample all minority class at once, we manually resample minority classes(less than 500 instances)
for i in range(minor_size):
    x_ada_train, y_ada_train = ada.fit_resample(x_ada_train, y_ada_train)
    
    
x_bdsmt_train, y_bdsmt_train = bdsmt.fit_resample(x_bdsmt_train, y_bdsmt_train)
x_smt_train, y_smt_train = smt.fit_resample(x_smt_train, y_smt_train)

#print("ada size: ", len(x_ada_train))
#print("bdsmt size: ", len(x_bdsmt_train))
print("ada size: ", len(y_ada_train))
print("bdsmt size: ", len(y_bdsmt_train))
print("smt size: ", len(y_smt_train))


counter = Counter(y_bdsmt_train)
for k,v in counter.items():
    per = v / len(y_bdsmt_train) * 100
    #print('Class=%d, n=%d (%.3f%%)' % (k, v, per))

ada size:  25076
bdsmt size:  22222
smt size:  23128


## Modelling

### xgboost

- option 1: XGBClassifier applies on BDLine_SMOTE data (sampling method)
    - BDLine_SMOTE perform the best
- option 2: XGBClassifier applies on unbalanced data with weight (cost sensitive method)
    - 2.1 scale_pos_weight


In [8]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import sklearn.metrics as metrics
import time

In [9]:
Test_Accuracy_Data = pd.DataFrame(columns = ['option','n_estimators','max_depth','random_state',
                                             'min_child_weight','gamma','learning_rate', 'eta'
                                             'subsample','colsample_bytree','n_jobs','Train ACC','Val ACC'])

In [ ]:
trainStartTime = time.time()

option = 'option 1'

# Number of trees
n_estimators = 50

# Maximum number of levels in tree
max_depth = 5

random_state = 0

#minimum sum of weights of all observations required in a child
min_child_weight = 2

#Gamma specifies the minimum loss reduction required to make a split
gamma = 0

# boosting learning rate
learning_rate = 0.01

subsample = 0.8 # 80% of data to grow trees and prevent overfitting

colsample_bytree = 0.80  # 85% of features used

eta = 0.1

n_jobs = -1



xgb = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth,
                    random_state = random_state, min_child_weight = min_child_weight,
                    gamma = gamma, learning_rate = learning_rate, subsample = subsample,
                    colsample_bytree = colsample_bytree, eta = eta, n_jobs = n_jobs)


model= xgb.fit(x_bdsmt_train, y_bdsmt_train)

Test_Accuracy_Data_One = pd.DataFrame(index = range(1),columns = ['option','n_estimators','max_depth','random_state',
                                                                  'min_child_weight','gamma','learning_rate', 'eta',
                                                                  'subsample','colsample_bytree','n_jobs',
                                                                  'Train ACC','Val ACC'])

Test_Accuracy_Data_One.loc[:,'option'] = option
Test_Accuracy_Data_One.loc[:,'n_estimators'] = n_estimators
Test_Accuracy_Data_One.loc[:,'max_depth'] = max_depth
Test_Accuracy_Data_One.loc[:,'random_state'] = random_state            
Test_Accuracy_Data_One.loc[:,'min_child_weight'] = min_child_weight
Test_Accuracy_Data_One.loc[:,'gamma'] = gamma
Test_Accuracy_Data_One.loc[:,'learning_rate'] = learning_rate       
Test_Accuracy_Data_One.loc[:,'eta'] = eta
Test_Accuracy_Data_One.loc[:,'subsample'] = subsample
Test_Accuracy_Data_One.loc[:,'colsample_bytree'] = colsample_bytree
Test_Accuracy_Data_One.loc[:,'n_jobs'] = n_jobs


train_pre = xgb.predict(x_bdsmt_train)
train_acc = xgb.score(x_bdsmt_train, y_bdsmt_train)
train_acc1 = accuracy_score(y_bdsmt_train, train_pre)

predictions = xgb.predict(x_val)
acc = xgb.score(x_val, y_val)
acc1 = accuracy_score(y_val, predictions)

Test_Accuracy_Data_One.loc[:,'Train ACC'] = train_acc
Test_Accuracy_Data_One.loc[:,'Val ACC'] = acc 

Test_Accuracy_Data = Test_Accuracy_Data.append(Test_Accuracy_Data_One)
            
print(train_acc1)
print(acc1)

trainFinishTime = time.time()
print("Time spent on training is: " + str(trainFinishTime - trainStartTime) + " sec")

Test_Accuracy_Data


## Output Final Result


In [ ]:
# test on the test data
predictions=xgb.predict(x_test)

with open("xgb_out.csv","w", newline="") as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(["Id","Category"])
    test_id = 0
    for pred in predictions:
        writer.writerow([test_id, pred])
        test_id+=1